In [ ]:
import os
import sys  
sys.path.insert(0, '/work/greenhouse-simulator-2/')

import math
import altair as alt
import pandas as pd
from pysolar import solar
from datetime import datetime, timezone
from IPython.core.display import display, HTML, Markdown, Image

from helpers.types import *
from helpers.data_prep import *
from helpers.visualization import *
from helpers.solar_conversions import *
from greenhouse.structure.materials import *
from greenhouse.structure.structure import *

In [ ]:
class Structure:
    def __init__(self,
        width=6,
        depth=2.6,
        front_height=2.5,
        rear_height=3,
        barrel_count=6,
        plants_per_barrel=200,
        barrel_diameter=0.57,
        barrel_height=1.5,
        barrel_rotator_power=10,
        latitude=38.7436883,
        longitude=-9.1952227,
        azimuth=180,
        roof_panel_type="polycarbonate"
    ):
        # Set up greenhouse primary parameters
        self.width: m = width
        self.depth: m = depth
        self.front_height: m = front_height
        self.rear_height: m = rear_height

        self.barrel_count = barrel_count
        self.plants_per_barrel = plants_per_barrel
        self.barrel_diameter: m = barrel_diameter
        self.barrel_height: m = barrel_height
        self.barrel_rotator_power: W = barrel_rotator_power

        # Set up location parameters
        self.coordinates = { "latitude": latitude, "longitude": longitude }
        self.azimuth = azimuth

        # Calculate derived parameters from primaries
        self.height_diff: m = self.rear_height - self.front_height
        self.roof_incline: deg = math.degrees(math.atan(self.height_diff / self.depth)) # Optimal for power generation when it equals to latitude

        self.barrel_circumference: m = math.pi * self.barrel_diameter
        self.barrel_surface_total: m2 = self.barrel_count * self.barrel_circumference * self.barrel_height
        self.barrel_surface_exposed_to_sun: m2 = self.barrel_surface_total / 3

        # Calculate surface areas (A) and volume (V)
        self.A_roof_panel: m2 = self.roof_panel_depth * self.width
        self.A_front_panel: m2 = self.width * self.front_height
        self.A_side_panel: m2 = self.get_side_panel_area()
        self.A_rear_panel: m2 = self.width * self.rear_height
        self.A_floor: m2 = self.width * self.depth

        is_roof_solar = False
        if roof_panel_type == "insolight":
            roof_panel = InsolightMaterial()
            is_roof_solar = True
        elif roof_panel_type == "solarbrite":
            roof_panel = SolarBriteMaterial()
            is_roof_solar = True
        elif roof_panel_type == "polycarbonate":
            roof_panel = Polycarbonate()
        else:
            raise Exception(f"Roof panel type '{roof_panel_type}' is not supported.")

        self.panels = {
            "slanted_roof": {
                "material": roof_panel,
                "area": self.A_roof_panel,
                "tilt": self.roof_incline,
                "azimuth_offset": 0,
                "solar": is_roof_solar
            },
            "front_panel": {
                "material": Polycarbonate(),
                "area": self.A_front_panel,
                "tilt": 90,
                "azimuth_offset": 0,
            },
            "left_panel": {
                "material": Polycarbonate(),
                "area": self.A_side_panel,
                "tilt": 90,
                "azimuth_offset": 90
            },
            "right_panel": {
                "material": Polycarbonate(),
                "area": self.A_side_panel,
                "tilt": 90,
                "azimuth_offset": -90
            },
            "rear_panel": {
                "material": Polycarbonate(),
                "area": self.A_rear_panel
            },
            "floor": {
                "material": Plywood(),
                "area": self.A_floor
            }
        }

        solar_panel_count = 0
        for panel_name in self.panels:
            if "solar" in self.panels[panel_name]:
                solar_panel_count += 1
        assert solar_panel_count == 1, "Only one structure panel can be solar."

    @property
    def irradiated_area(self):
        irradiated_area: m2 = 0
        for panel_name in self.panels:
            if "solar" in self.panels[panel_name]:
                irradiated_area += self.panels[panel_name]["area"]

        return irradiated_area

    @property
    def power_consumption(self) -> W:
        """
        Get electric power consumption of the structure. Includes: barrel rotators.
        """
        return self.barrel_count * self.barrel_rotator_power


    @property
    def roof_panel_depth(self) -> m:
        """
        Calculate depth (side edge if observing from front) of roof panel using Pythagoras' theorem
        """
        tan_alpha = self.height_diff / self.depth
        alpha: rad = math.atan(tan_alpha)

        return self.height_diff / math.sin(alpha)

    @property
    def volume(self) -> m3:
        """
        Get total air volume of the greenhouse, including an estimated offset accounting for the equipment inside.
        """
        offset = 0.98
        V_triangle: m3 = (self.height_diff * self.depth) / 2 * self.width
        V_rectangle: m3 = self.depth * self.front_height * self.width
        V_total: m3 = V_triangle + V_rectangle

        return V_total * offset
    

    def get_side_panel_area(self) -> m2:
        """
        Calculate area of trapeziod-shaped side panels.
        """
        A_triangle: m2 = (self.height_diff * self.depth) / 2
        A_rectangle: m2 = self.depth * self.front_height

        return A_triangle + A_rectangle


    def get_heat_transfer_rate(self, delta_T: C) -> W:
        """
        Get the heat transfer rate through the structure.
        """
        heat_transfer_rate: W = 0
        for panel_name in self.panels:
            panel = self.panels[panel_name]
            U: W_per_m2_K = panel["material"].U_value
            A: m2 = panel["area"]
            heat_transfer_rate_of_panel: W = U * A * delta_T
            heat_transfer_rate += heat_transfer_rate_of_panel

        return heat_transfer_rate


    def get_irradiance_on_panels(self, timestamp, irradiance: W_per_m2):
        """
        Get solar irradiance on each panel.

        Returns
        -------
        irradiance_on_panels : dict
            Solar irradiance for each panel per square meters.
        """
        irradiance_on_panels = {}
        for panel_name in self.panels:
            panel = self.panels[panel_name]
            if "tilt" in panel: # To filter out panels that are not reached by sunlight
                intensity_coeff_of_panel = get_intensity_coeff(self.coordinates, timestamp, panel["tilt"], self.azimuth + panel["azimuth_offset"])
                irradiance_on_panels[panel_name] = intensity_coeff_of_panel * irradiance

        return irradiance_on_panels


    def get_irradiance_by_panel_type(self, timestamp, irradiance: W_per_m2) -> [W, W_per_m2]:
        irradiance_on_panels = self.get_irradiance_on_panels(timestamp, irradiance)

        solar_power_on_nonsolar_panels: W = 0
        irradiance_on_solar_panels: W_per_m2 = 0
        for panel_name in irradiance_on_panels:
            solar_power_on_panel: W = irradiance_on_panels[panel_name] * self.panels[panel_name]["area"] * self.panels[panel_name]["material"].transparency
            if "solar" in self.panels[panel_name]:
                irradiance_on_solar_panels = irradiance_on_panels[panel_name]
            else:
                solar_power_on_nonsolar_panels += solar_power_on_panel

        return solar_power_on_nonsolar_panels, irradiance_on_solar_panels


### Test

In [ ]:
structure = Structure(
    width=10,
    depth=2,
    front_height=3,
    rear_height=4,
    barrel_count=4,
    plants_per_barrel=200,
    barrel_diameter=0.5,
    barrel_height=1.5,
    barrel_rotator_power=0
)

assert round(structure.roof_panel_depth, 3) == 2.236, "Error when calculating 'roof_panel_depth'"
assert round(structure.barrel_surface_total, 3) == 9.425, "Error when calculating 'barrel_surface_total'"
assert round(structure.roof_incline, 3) == 26.565, "Error when calculating 'roof_incline'"

### Visualize

In [ ]:
structure = Structure(
    width=6,
    depth=2.6,
    front_height=2.5,
    rear_height=3.5,
    barrel_count=4,
    plants_per_barrel=200,
    barrel_diameter=0.57,
    barrel_height=1.5,
    barrel_rotator_power=10,
    latitude=38.7436883,
    longitude=-9.1952227,
    azimuth=180
)

Iterate through dataset and add irradiance data for each panel and total for the whole greenhouse

In [ ]:
if not os.environ.get("USING_RUN"):
    df = get_weather_data(date_from="2020-07-02", date_to="2020-07-03", resample_period="15min")

    for i, row in df.iterrows():
        irradiance_on_panels = structure.get_irradiance_on_panels(i, row["solarradiation"])
        solar_power_on_solar_panel, solar_power_on_nonsolar_panel = structure.get_irradiance_by_panel_type(i, row["solarradiation"])

        df.loc[i, "solar_power_on_solar_panel"] = solar_power_on_solar_panel
        df.loc[i, "solar_power_on_nonsolar_panel"] = solar_power_on_nonsolar_panel
        df.loc[i, "slanted_roof_irradiance"] = irradiance_on_panels["slanted_roof"]
        df.loc[i, "front_panel_irradiance"] = irradiance_on_panels["front_panel"]
        df.loc[i, "left_panel_irradiance"] = irradiance_on_panels["left_panel"]
        df.loc[i, "right_panel_irradiance"] = irradiance_on_panels["right_panel"]


**TOP**: Plot the solar irradiance per square meter on each panel as well as the incident irradiance (solarradiation)

**BOTTOM**: Plot total irradiance reaching the greenhouse

In [ ]:
plots = None
if not os.environ.get("USING_RUN"):
    multiline_cols = ["solarradiation", "slanted_roof_irradiance", "front_panel_irradiance", "left_panel_irradiance", "right_panel_irradiance"]
    plots = plot_multiline(df, multiline_cols, width=900, height=235, y_label="W / m2", legend_label="") & \
    plot_multiline(df, ["solar_power_on_solar_panel", "solar_power_on_nonsolar_panel"], width=900, height=235, y_label="W")
plots

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=2f5dc715-67f7-4c8c-98f7-a87b736d3338' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>